In [1]:
from curriculum import get_cbc_grouped_questions, get_rubrics_by_sub_strand
from utils import *

## Generate questions

In [2]:
grouped_questions = get_cbc_grouped_questions(
    strand_ids=[8,9,10], 
    question_count=5,
    bloom_skill_count=1,
    is_debug=True, 
    curriculum_file="data/cbc_data.json",
)


✅ Question breakdown written to output/question_breakdown.json. Total: 3


In [3]:
all_question_list = generate_llm_question_list(
    grouped_question_data=grouped_questions,
    is_debug=True,
)

# If there was a generation error
if not isinstance(all_question_list, list):
    print(all_question_list)


Machines =========
📝 Input token count (gpt-4o): 715
📦 Raw LLM output:
 content='[{"question":"In a school science project, Amina uses a lever to lift a heavy stone. Explain how the lever makes her work easier and identify the type of simple machine a lever is.","expected_answer":"A lever makes work easier by allowing Amina to apply less force over a greater distance to lift the stone, and it is a type of simple machine known as a lever."}]' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 722, 'total_tokens': 802, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_90122d973c', 'id': 'chatcmpl-BWh9O92COKTJptyFHrRXfKt6SfGHg', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--928d617

In [4]:
exam_questions = get_db_question_objects(
    all_question_list=all_question_list,
    is_debug=True,
)


✅ Question list to output/question_list.json. Total: 5


## Simulate student answers

In [5]:
def convert_to_single_qa_list(data):
    result = []
    for index, item in enumerate(data):
        questions = item["questions"]
        expected_answers = item["expected_answers"]

        if isinstance(questions, str):
            questions = [questions]
        if isinstance(expected_answers, str):
            expected_answers = [expected_answers]

        if questions and expected_answers:
            result.append({
                "id": index + 1,
                "question": questions[0],
                "expected_answer": expected_answers[0]
            })

    return result

exam_output = convert_to_single_qa_list(exam_questions)
print(exam_output)

[{'id': 1, 'question': "Amina noticed that her family's soap does not lather well when she uses water from the well, but it lathers easily with rainwater. What is the difference between the well water and rainwater that affects the soap's ability to lather?", 'expected_answer': 'The well water is likely hard water, which contains minerals like calcium and magnesium that prevent soap from lathering well, while rainwater is soft water and does not contain these minerals.'}, {'id': 2, 'question': 'What are the three main types of pollution commonly found in urban areas like Nairobi?', 'expected_answer': 'The three main types of pollution commonly found in urban areas like Nairobi are air pollution, water pollution, and noise pollution.'}, {'id': 3, 'question': 'In a school science project, Amina uses a lever to lift a heavy stone. Explain how the lever makes her work easier and identify the type of simple machine a lever is.', 'expected_answer': 'A lever makes work easier by allowing Amin

In [6]:
sample_students = [{'id': 1, 'avg_score': 12}, {'id': 2, 'avg_score': 95}, {
    'id': 3, 'avg_score': 54}, {'id': 4, 'avg_score': 71},]

In [7]:
parsed_answers = generate_llm_exam_answers_list(
    llm=OPENAI_LLM_4O,
    exam_data=exam_output,
    student_data=sample_students,
    is_debug=True,
)

📝 Input token count (gpt-4o): 951
📦 Raw LLM output:
 content='```json\n[\n  {\n    "id": 1,\n    "answers": [\n      {\n        "question_id": "1",\n        "answer": "I think the well water has something in it that stops the soap from making bubbles, but rainwater doesn\'t have that."\n      },\n      {\n        "question_id": "2",\n        "answer": "In Nairobi, there is a lot of smoke from cars, dirty water, and loud noises."\n      },\n      {\n        "question_id": "3",\n        "answer": "A lever helps lift things by making it easier to push up the stone."\n      },\n      {\n        "question_id": "4",\n        "answer": "Boiling the water makes it better for soap because it changes something in the water."\n      },\n      {\n        "question_id": "5",\n        "answer": "Cleaning up helps make the place look better, but it might not stop all the pollution."\n      }\n    ]\n  },\n  {\n    "id": 2,\n    "answers": [\n      {\n        "question_id": "1",\n        "answer": "We

In [8]:
# # ==== DB MOCK ANSWERS
# # ====================
# exam = []
# EXAM_FILE = "data/exam.json"

# with open(EXAM_FILE, "r") as f:
#     exam = json.load(f)

# print(exam)


# students = []
# STUDENT_FILE = "data/classroom.json"

# with open(STUDENT_FILE, "r") as f:
#     students = json.load(f)

# print(students)


# parsed_answers = generate_llm_exam_answers_list(
#     llm=OPENAI_LLM_4O,
#     exam_data=exam,
#     student_data=students,
#     is_debug=True,
# )

## Grade answers

In [9]:
def get_answers_for_question(target_question_id, start_count):
    result = []
    count = start_count
    for entry in parsed_answers:
        for answer in entry["answers"]:
            if answer["question_id"] == str(target_question_id):
                result.append({
                    "answer_id": count,
                    "answer": answer["answer"],
                })
                count += 1
    return result, count


grouped_answers_data = []
count = 1

for idx, q in enumerate(exam_questions):
    item = {
        "question": q["description"],
        "expected_answer": q["expected_answer"],
        "rubrics": get_rubrics_by_sub_strand(
            sub_strand_name=q["sub_strand"],
            curriculum_file="data/cbc_data.json",
        )
    }
    student_answers, count = get_answers_for_question(idx + 1, count)
    item["student_answers"] = student_answers

    grouped_answers_data.append(item)

In [10]:
ANSWERS_LIST_OUTPUT_FILE = "output/answers_list.json"
with open(ANSWERS_LIST_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(grouped_answers_data, f, ensure_ascii=False, indent=4)
print(f"✅ Mocked Answers list written to {ANSWERS_LIST_OUTPUT_FILE}")

✅ Mocked Answers list written to output/answers_list.json


In [11]:
parsed_grades = generate_llm_answer_grades_list(
    llm=OPENAI_LLM_4O,
    grouped_answers_data=grouped_answers_data,
    is_debug=True,
)


Amina noticed that her family's soap does not lather well when she uses water from the well, but it lathers easily with rainwater. What is the difference between the well water and rainwater that affects the soap's ability to lather? =========
📝 Input token count (gpt-4o): 654
📦 Raw LLM output:
 content='```json\n[\n    {"answer_id": 1, "score": 2},\n    {"answer_id": 2, "score": 3},\n    {"answer_id": 3, "score": 2},\n    {"answer_id": 4, "score": 3}\n]\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 661, 'total_tokens': 724, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BWh9lQxE4lPXGwQ6S13DqPTxMBr7H', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': Non